In [45]:
!pip install transformers datasets wandb

In [ ]:
import torch

torch.cuda.is_available()

In [ ]:
from datasets import load_dataset

import pandas as pd

df = pd.read_csv('Annotated_story_pairs2.csv')
df['text'] = '#Sentence:' + df['Shared_stories'] + '#Sentence:' + df['Response_strories']
df['label'] = df['Rate']

df[['text', 'label']].to_csv('Annotated_story_pairs3.csv', index=False)

dataset = load_dataset("csv", data_files="Annotated_story_pairs3.csv")

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


NameError: name 'dataset' is not defined

In [49]:
ds = tokenized_dataset['train'].train_test_split(test_size=0.1, shuffle=True, seed=42)
ds_test = ds['test']
ds_train = ds['train'].train_test_split(test_size=0.15, shuffle=True, seed=42)
ds_val = ds_train['test']
ds_train = ds_train['train']

In [50]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
%env WANDB_PROJECT = Empathy_project

env: WANDB_PROJECT=Empathy_project


In [52]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='Stories10',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
'''metrics = trainer.evaluate()
print(metrics)'''

In [ ]:
# predictions1 = trainer.predict(ds_test)
